In [ ]:
"""Using 'all' file instead of 'wmo' file"""
import numpy as np 
import pandas as pd
import pickle as pkl
from netCDF4 import Dataset,num2date
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import norm, poisson, lognorm, chisquare, linregress, ttest_ind, power_divergence, ks_2samp, chi2_contingency
from scipy.interpolate import interp1d
from scipy.special import factorial
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
#%matplotlib tk    #Uncomment for interactive figures
import geopy.distance as gd
from sklearn.metrics import mean_squared_error
import time
import os

In [ ]:
tic = time.perf_counter()
path_to_data = os.path.join('data')

ibt = dict() 
#loaded variables from cell 2 of Year3Projectfinal.ipynb which are used later in code
#from ibtracs database and etopo1 global relief model

for filename in os.listdir(path_to_data):
    if filename.endswith('.pickle'):
        path_to_file = os.path.join(path_to_data, filename)
        with open(path_to_file,'rb') as f:
            ibt[filename[:-7]] = pkl.load(f) # d[key] = value, where key is filename str w/o '.pickle'
            #expected usage example: ibtracs_and_landmask['lat']
toc = time.perf_counter()
print(toc-tic)

In [ ]:
#Extract PHI data
tic = time.time() #Expect runtime ~3min

#Input extreme point = [x,y]
extremeP_N = [122.9, 19.2]
extremeP_S = [122.6, 3.5]
extremeP_E = [128.2, 6.2]
extremeP_W = [117.4, 18.1]

#Slope of lines bounding region
m_top_P = (extremeP_N[1]-extremeP_W[1])/(extremeP_N[0]-extremeP_W[0])
m_bot_P = (extremeP_S[1]-extremeP_E[1])/(extremeP_S[0]-extremeP_E[0])
m_left_P = (extremeP_S[1]-extremeP_W[1])/(extremeP_S[0]-extremeP_W[0])
m_right_P = (extremeP_N[1]-extremeP_E[1])/(extremeP_N[0]-extremeP_E[0])

#Intercept of lines
c_top_P = extremeP_N[1]-m_top_P*extremeP_N[0]
c_bot_P = extremeP_S[1]-m_bot_P*extremeP_S[0]
c_left_P = extremeP_S[1]-m_left_P*extremeP_S[0]
c_right_P = extremeP_N[1]-m_right_P*extremeP_N[0]

#Get points in land mask file which are in PHI area
def philippine_landmask_points(landmask_LON,landmask_LAT):
    landmask_points = []
    
    for i in range(17400,18500): 
        for i2 in range(5600,6600):
            equation_top = m_top_P * landmask_LON[i,i2] + c_top_P
            equation_bot = m_bot_P * landmask_LON[i,i2] + c_bot_P 
            equation_left = m_left_P * landmask_LON[i,i2] + c_left_P 
            equation_right  = m_right_P * landmask_LON[i,i2] + c_right_P
    
            if landmask_LAT[i,i2] <= equation_top and landmask_LAT[i,i2] <= equation_right and landmask_LAT[i,i2] >= equation_bot and landmask_LAT[i,i2] >= equation_left:
                landmask_points.append(i)
                landmask_points.append(i2)
        
    return landmask_points
    
#Get TC centre points in PHI area
def philippine_data_extraction(LON,LAT,after2014=False,separate_NS=False): 
    #after2014: True means only include TC data after 2014
    #Separate NS: If you want to separate PHI into Northern and Southern parts, type 'N' or 'S' respectively,
    #    otherwise, or for GD, type 'False'
    
    philippine_data = []
    PHI_TCposition = {}
    
    philippine_data_N = []
    philippine_data_S = []
    PHI_TCposition_N = {}
    PHI_TCposition_S = {}
    
    if separate_NS == False:
        for i in range (len(LON)):
            PHI_TCposition[str(i)] = []
            for i2 in range (len(LON[i])): 
                equation_top = m_top_P * LON[i,i2] + c_top_P
                equation_bot = m_bot_P * LON[i,i2] + c_bot_P 
                equation_left = m_left_P * LON[i,i2] + c_left_P 
                equation_right  = m_right_P * LON[i,i2] + c_right_P

                if after2014 == True:
                    if i in TCinseason and LAT[i,i2] <= equation_top and LAT[i,i2] <= equation_right and LAT[i,i2] >= equation_bot and LAT[i,i2] >= equation_left: 
                        philippine_data.append((i))
                        PHI_TCposition[str(i)].append(i2)
                else:
                    if LAT[i,i2] <= equation_top and LAT[i,i2] <= equation_right and LAT[i,i2] >= equation_bot and LAT[i,i2] >= equation_left: 
                        philippine_data.append((i))
                        PHI_TCposition[str(i)].append(i2)
                        
        return np.unique(philippine_data), PHI_TCposition
                    
    elif separate_NS == True:
        NSlatboundary = 14.4 #Set latitude for dividing into North and South
        for i in range(len(LON)):
            PHI_TCposition_N[str(i)] = []
            PHI_TCposition_S[str(i)] = []
            for i2 in range (len(LON[i])): 
                equation_top = m_top_P * LON[i,i2] + c_top_P #Equation of straight line
                equation_bot = m_bot_P * LON[i,i2] + c_bot_P 
                equation_left = m_left_P * LON[i,i2] + c_left_P 
                equation_right  = m_right_P * LON[i,i2] + c_right_P
                
                #Check to see if a point is inside the boundary
                if after2014 == True:
                    if i in TCinseason and LAT[i,i2] > NSlatboundary and LAT[i,i2] <= equation_top and LAT[i,i2] <= equation_right and LAT[i,i2] >= equation_bot and LAT[i,i2] >= equation_left: 
                        philippine_data_N.append((i))
                        PHI_TCposition_N[str(i)].append(i2)
                    elif i in TCinseason and LAT[i,i2] <= NSlatboundary and LAT[i,i2] <= equation_top and LAT[i,i2] <= equation_right and LAT[i,i2] >= equation_bot and LAT[i,i2] >= equation_left: 
                        philippine_data_S.append((i))
                        PHI_TCposition_S[str(i)].append(i2)

                else:
                    if LAT[i,i2] > NSlatboundary and LAT[i,i2] <= equation_top and LAT[i,i2] <= equation_right and LAT[i,i2] >= equation_bot and LAT[i,i2] >= equation_left: 
                        philippine_data_N.append((i))
                        PHI_TCposition_N[str(i)].append(i2)
                    elif LAT[i,i2] <= NSlatboundary and LAT[i,i2] <= equation_top and LAT[i,i2] <= equation_right and LAT[i,i2] >= equation_bot and LAT[i,i2] >= equation_left: 
                        philippine_data_S.append((i))
                        PHI_TCposition_S[str(i)].append(i2)
                    
        return np.unique(philippine_data_N), np.unique(philippine_data_S), PHI_TCposition_N, PHI_TCposition_S
                

landmask_in_area_PHI = np.array(philippine_landmask_points(landmask_LON,landmask_LAT))
landmask_in_area_PHI = landmask_in_area_PHI.reshape(int(len(landmask_in_area_PHI)/2),2)

z_coastline_PHI = [] #Find PHI coastline from land mask (this list contains index of elements)
z_land_PHI = [] #Find PHI land points from land mask
for i in range(len(landmask_in_area_PHI)):
    if z[landmask_in_area_PHI[i,1],landmask_in_area_PHI[i,0]] == 0: #Coast if altitude == 0
        z_coastline_PHI.append(landmask_in_area_PHI[i,0])
        z_coastline_PHI.append(landmask_in_area_PHI[i,1])
    elif z[landmask_in_area_PHI[i,1],landmask_in_area_PHI[i,0]] >= 0: #Land if altitude >= 0
        z_land_PHI.append(landmask_in_area_PHI[i,0])
        z_land_PHI.append(landmask_in_area_PHI[i,1])
        
z_coastline_PHI = np.asarray(z_coastline_PHI).reshape(int(len(z_coastline_PHI)/2),2)
z_land_PHI = np.asarray(z_land_PHI).reshape(int(len(z_land_PHI)/2),2)

z_coastline_coord_PHI = np.zeros(z_coastline_PHI.shape) #list of coordinates of PHI coastline (change index to actual coordinates)
z_coastline_coord_PHI[:,0] = landmask_lon[z_coastline_PHI[:,0]]
z_coastline_coord_PHI[:,1] = landmask_lat[z_coastline_PHI[:,1]]

z_land_coord_PHI = np.zeros(z_land_PHI.shape) #List of coordinates of PHI land
z_land_coord_PHI[:,0] = landmask_lon[z_land_PHI[:,0]]
z_land_coord_PHI[:,1] = landmask_lat[z_land_PHI[:,1]]

landfallTC_PHI_jtwc = philippine_data_extraction(LON[:,:,0],LAT[:,:,0])[0] #list (keys)
landfallTC_PHI_jtwc_after2014 = philippine_data_extraction(LON[:,:,0],LAT[:,:,0], True)[0]
landfallTC_PHI_cma = philippine_data_extraction(LON[:,:,1],LAT[:,:,1])[0] #list (keys)
landfallTC_PHI_cma_after2014 = philippine_data_extraction(LON[:,:,1],LAT[:,:,1], True)[0]
landfallTC_PHI_wmo = philippine_data_extraction(LON[:,:,2],LAT[:,:,2])[0] #list (keys)
landfallTC_PHI_wmo_after2014 = philippine_data_extraction(LON[:,:,2],LAT[:,:,2], True)[0]

PHI_TCposition_jtwc = philippine_data_extraction(LON[:,:,0],LAT[:,:,0])[1] #dict (tc index and position index)
PHI_TCposition_jtwc_after2014 = philippine_data_extraction(LON[:,:,0],LAT[:,:,0],True)[1]
PHI_TCposition_cma = philippine_data_extraction(LON[:,:,1],LAT[:,:,1])[1] #dict (tc index and position index)
PHI_TCposition_cma_after2014 = philippine_data_extraction(LON[:,:,1],LAT[:,:,1],True)[1]
PHI_TCposition_wmo = philippine_data_extraction(LON[:,:,2],LAT[:,:,2])[1] #dict (tc index and position index)
PHI_TCposition_wmo_after2014 = philippine_data_extraction(LON[:,:,2],LAT[:,:,2],True)[1]

landfallTC_PHI_jtwc_N, landfallTC_PHI_jtwc_S, PHI_TCposition_jtwc_N, PHI_TCposition_jtwc_S = philippine_data_extraction(LON[:,:,0],LAT[:,:,0],False, True)
landfallTC_PHI_jtwc_N_after2014, landfallTC_PHI_jtwc_S_after2014, PHI_TCposition_jtwc_N_after2014, PHI_TCposition_jtwc_S_after2014 = philippine_data_extraction(LON[:,:,0],LAT[:,:,0],True, True)
landfallTC_PHI_cma_N, landfallTC_PHI_cma_S, PHI_TCposition_cma_N, PHI_TCposition_cma_S = philippine_data_extraction(LON[:,:,1],LAT[:,:,1],False, True)
landfallTC_PHI_cma_N_after2014, landfallTC_PHI_cma_S_after2014, PHI_TCposition_cma_N_after2014, PHI_TCposition_cma_S_after2014 = philippine_data_extraction(LON[:,:,1],LAT[:,:,1],True, True)
landfallTC_PHI_wmo_N, landfallTC_PHI_wmo_S, PHI_TCposition_wmo_N, PHI_TCposition_wmo_S = philippine_data_extraction(LON[:,:,2],LAT[:,:,2],False, True)
landfallTC_PHI_wmo_N_after2014, landfallTC_PHI_wmo_S_after2014, PHI_TCposition_wmo_N_after2014, PHI_TCposition_wmo_S_after2014 = philippine_data_extraction(LON[:,:,2],LAT[:,:,2],True, True)

toc = time.time()
print(toc - tic)

In [ ]:
#Extract GD data

tic = time.time() #Expect runtime ~2min

extremeG_N = [117.4, 24.2]
extremeG_S = [109.8, 19.9]
extremeG_E = [117.7, 22.9]
extremeG_W = [109.4, 22.2]

m_top_G = (extremeG_N[1]-extremeG_W[1])/(extremeG_N[0]-extremeG_W[0])
m_bot_G = (extremeG_S[1]-extremeG_E[1])/(extremeG_S[0]-extremeG_E[0])
m_left_G = (extremeG_S[1]-extremeG_W[1])/(extremeG_S[0]-extremeG_W[0])
m_right_G = (extremeG_N[1]-extremeG_E[1])/(extremeG_N[0]-extremeG_E[0])

c_top_G = extremeG_N[1]-m_top_G*extremeG_N[0]
c_bot_G = extremeG_S[1]-m_bot_G*extremeG_S[0]
c_left_G = extremeG_S[1]-m_left_G*extremeG_S[0]
c_right_G = extremeG_N[1]-m_right_G*extremeG_N[0]

def guangdong_landmask_points(landmask_LON,landmask_LAT):

    landmask_points = []

    for i in range(17200,17900): 
        for i2 in range(6500,7000):
            equation_top = m_top_G * landmask_LON[i,i2] + c_top_G
            equation_bot = m_bot_G * landmask_LON[i,i2] + c_bot_G 
            equation_left = m_left_G * landmask_LON[i,i2] + c_left_G
            equation_right  = m_right_G * landmask_LON[i,i2] + c_right_G
    
            if landmask_LAT[i,i2] <= equation_top and landmask_LAT[i,i2] <= equation_right and landmask_LAT[i,i2] >= equation_bot and landmask_LAT[i,i2] >= equation_left:
                landmask_points.append(i)
                landmask_points.append(i2)
        
    return landmask_points

def guangdong_data_extraction(LON,LAT,after2014=False): 
    guangdong_data = []
    GD_TCposition = {}
    
    for i in range (len(LON)):
        GD_TCposition[str(i)] = []
        for i2 in range (len(LON[i])):
            equation_top = m_top_G * LON[i,i2] + c_top_G
            equation_bot = m_bot_G * LON[i,i2] + c_bot_G 
            equation_left = m_left_G * LON[i,i2] + c_left_G 
            equation_right  = m_right_G * LON[i,i2] + c_right_G
            
            if after2014 == True:
                if i in TCinseason and LAT[i,i2] <= equation_top and LAT[i,i2] <= equation_right and LAT[i,i2] >= equation_bot and LAT[i,i2] >= equation_left: 
                    guangdong_data.append((i))
                    GD_TCposition[str(i)].append(i2)
            else:
                if LAT[i,i2] <= equation_top and LAT[i,i2] <= equation_right and LAT[i,i2] >= equation_bot and LAT[i,i2] >= equation_left: 
                    guangdong_data.append((i))
                    GD_TCposition[str(i)].append(i2)
            
    return np.unique(guangdong_data), GD_TCposition

landmask_in_area_GD = np.asarray(guangdong_landmask_points(landmask_LON,landmask_LAT))
landmask_in_area_GD = landmask_in_area_GD.reshape(int(len(landmask_in_area_GD)/2),2)


z_coastline_GD = []
z_land_GD = []
for i in range(len(landmask_in_area_GD)):
    if z[landmask_in_area_GD[i,1],landmask_in_area_GD[i,0]] == 0: 
        z_coastline_GD.append(landmask_in_area_GD[i,0])
        z_coastline_GD.append(landmask_in_area_GD[i,1])
    elif z[landmask_in_area_GD[i,1],landmask_in_area_GD[i,0]] >= 0:
        z_land_GD.append(landmask_in_area_GD[i,0])
        z_land_GD.append(landmask_in_area_GD[i,1])
        
z_coastline_GD = np.asarray(z_coastline_GD).reshape(int(len(z_coastline_GD)/2),2)
z_land_GD = np.asarray(z_land_GD).reshape(int(len(z_land_GD)/2),2)

z_coastline_coord_GD = np.zeros(z_coastline_GD.shape) #list of coordinates of GD coastline
z_coastline_coord_GD[:,0] = landmask_lon[z_coastline_GD[:,0]]
z_coastline_coord_GD[:,1] = landmask_lat[z_coastline_GD[:,1]]

z_land_coord_GD = np.zeros(z_land_GD.shape)
z_land_coord_GD[:,0] = landmask_lon[z_land_GD[:,0]]
z_land_coord_GD[:,1] = landmask_lat[z_land_GD[:,1]]
        
landfallTC_GD_jtwc = guangdong_data_extraction(LON[:,:,0],LAT[:,:,0])[0] #list (index)
landfallTC_GD_jtwc_after2014 = guangdong_data_extraction(LON[:,:,0],LAT[:,:,0],True)[0]
landfallTC_GD_cma = guangdong_data_extraction(LON[:,:,1],LAT[:,:,1])[0] #list (index)
landfallTC_GD_cma_after2014 = guangdong_data_extraction(LON[:,:,1],LAT[:,:,1],True)[0]
landfallTC_GD_wmo = guangdong_data_extraction(LON[:,:,2],LAT[:,:,2])[0] #list (index)
landfallTC_GD_wmo_after2014 = guangdong_data_extraction(LON[:,:,2],LAT[:,:,2],True)[0]

GD_TCposition_jtwc = guangdong_data_extraction(LON[:,:,0],LAT[:,:,0])[1] #dict (tc index & position index)
GD_TCposition_jtwc_after2014 = guangdong_data_extraction(LON[:,:,0],LAT[:,:,0],True)[1]
GD_TCposition_cma = guangdong_data_extraction(LON[:,:,1],LAT[:,:,1])[1] #dict (tc index & position index)
GD_TCposition_cma_after2014 = guangdong_data_extraction(LON[:,:,1],LAT[:,:,1],True)[1]
GD_TCposition_wmo = guangdong_data_extraction(LON[:,:,2],LAT[:,:,2])[1] #dict (tc index & position index)
GD_TCposition_wmo_after2014 = guangdong_data_extraction(LON[:,:,2],LAT[:,:,2],True)[1]

toc = time.time()
print(toc - tic)

In [ ]:
#----------------------------------------------------
'''
Here, we are only looking at the landfall typhoon point and the point before that
'''

tic = time.time() #Expected runtime ~45s

#Gets the data for TCs at first landfall point, and closest point before landfall
def get_landfall_data_beforeandafter(area, source, after2014 = False, PHI_NS = False):
    #area: if PHI, type '0'; if GD, type '1'
    #source: if JTWC, type '0'; if CMA, type '1'; if WMO, type '2'
    #after2014: True means only include TC data after 2014
    #Separate NS: If you want to separate PHI into Northern and Southern parts, type 'N' or 'S' respectively,
    #    otherwise, or for GD, type 'False'
    
    pres = {}
    wind = {}
    coord = {}
    disttoland = {}
    list_of_index = {}
    epsilon = 1e-5 #To account for rounding errors when comparing numbers
    
    if area == 0: #PHI
        if source == 0: #jtwc
            if PHI_NS == False:
                if after2014 == True:
                    tcindex = landfallTC_PHI_jtwc_after2014
                    posindex = PHI_TCposition_jtwc_after2014
                else:
                    tcindex = landfallTC_PHI_jtwc
                    posindex = PHI_TCposition_jtwc
            elif PHI_NS == 'N':
                if after2014 == True:
                    tcindex = landfallTC_PHI_jtwc_N_after2014
                    posindex = PHI_TCposition_jtwc_N_after2014
                else:
                    tcindex = landfallTC_PHI_jtwc_N
                    posindex = PHI_TCposition_jtwc_N
            elif PHI_NS == 'S':
                if after2014 == True:
                    tcindex = landfallTC_PHI_jtwc_S_after2014
                    posindex = PHI_TCposition_jtwc_S_after2014
                else:
                    tcindex = landfallTC_PHI_jtwc_S
                    posindex = PHI_TCposition_jtwc_S
        elif source == 1: #cma
            if PHI_NS == False:
                if after2014 == True:
                    tcindex = landfallTC_PHI_cma_after2014
                    posindex = PHI_TCposition_cma_after2014
                else:
                    tcindex = landfallTC_PHI_cma
                    posindex = PHI_TCposition_cma
            elif PHI_NS == 'N':
                if after2014 == True:
                    tcindex = landfallTC_PHI_cma_N_after2014
                    posindex = PHI_TCposition_cma_N_after2014
                else:
                    tcindex = landfallTC_PHI_cma_N
                    posindex = PHI_TCposition_cma_N
            elif PHI_NS == 'S':
                if after2014 == True:
                    tcindex = landfallTC_PHI_cma_S_after2014
                    posindex = PHI_TCposition_cma_S_after2014
                else:
                    tcindex = landfallTC_PHI_cma_S
                    posindex = PHI_TCposition_cma_S
        elif source == 2: #wmo
            if PHI_NS == False:
                if after2014 == True:
                    tcindex = landfallTC_PHI_wmo_after2014
                    posindex = PHI_TCposition_wmo_after2014
                else:
                    tcindex = landfallTC_PHI_wmo
                    posindex = PHI_TCposition_wmo
            elif PHI_NS == 'N':
                if after2014 == True:
                    tcindex = landfallTC_PHI_wmo_N_after2014
                    posindex = PHI_TCposition_wmo_N_after2014
                else:
                    tcindex = landfallTC_PHI_wmo_N
                    posindex = PHI_TCposition_wmo_N
            elif PHI_NS == 'S':
                if after2014 == True:
                    tcindex = landfallTC_PHI_wmo_S_after2014
                    posindex = PHI_TCposition_wmo_S_after2014
                else:
                    tcindex = landfallTC_PHI_wmo_S
                    posindex = PHI_TCposition_wmo_S
            
            
        landmask_lat = z_land_coord_PHI[:,1]
        landmask_lon = z_land_coord_PHI[:,0]
        for i in list(tcindex): #Loop over each TC in list
            landindexinarea = []
            for j in posindex[str(i)]: #Loop over each data point for that TC
                TClat = LAT[:,:,source][i][j]
                TClon = LON[:,:,source][i][j]
                
                #Find all land points with same latitude
                landpoint_onlat_temp1 = np.where(landmask_lat <= TClat + epsilon)
                landpoint_onlat_temp2 = np.where(landmask_lat >= TClat - epsilon)
                landpoint_onlat_index = np.intersect1d(landpoint_onlat_temp1,landpoint_onlat_temp2) #Index of points in landmask array where latitude is same as TClat
                landpoint_onlat_lon = landmask_lon[landpoint_onlat_index] #Lon of landmask points with same lat as TClat
                
                if landpoint_onlat_lon.size != 0:
                    landpoint_onlat_maxlon = max(landpoint_onlat_lon) #Find easternmost PHI land point on same latitude
                    landpoint_onlat_minlon = min(landpoint_onlat_lon) #Find westernmost PHI land point on same latitude
                
                    #Check whether TC is on land or ocean
                    if landpoint_onlat_minlon-epsilon <= TClon <= landpoint_onlat_maxlon+epsilon:
                        landindexinarea.append(j)
            
            landindexinarea = np.asarray(landindexinarea)
            if landindexinarea.size != 0:
                first_land_index = landindexinarea[0]
                if first_land_index == 0:
                    indexlist = [first_land_index]
                else: 
                    indexlist = [first_land_index-1, first_land_index]

                pres[str(i)] = MIN_PRES[:,:,source][int(i)][indexlist]
                pres[str(i)] = [k for k in pres[str(i)] if k >= 0]
                if pres[str(i)] == []:
                    del pres[str(i)]
                wind[str(i)] = MAX_WIND[:,:,source][int(i)][indexlist]
                wind[str(i)] = [k for k in wind[str(i)] if k >= 0]
                if wind[str(i)] == []:
                    del wind[str(i)]
                coord[str(i)] = np.array([LAT[:,:,source][int(i),indexlist],LON[:,:,source][int(i),indexlist]]).transpose()
                if list(coord[str(i)][0]) == [-3e4,-3e4]: #Remove non-existant data
                    coord[str(i)] = np.delete(coord[str(i)],0,0)
                disttoland[str(i)] = DIST2LAND[int(i)][indexlist]
                list_of_index[str(i)] = indexlist

        if source == 1 and (PHI_NS == False or PHI_NS == 'N'):
            #Add Mangkhut (to the end of the dict), source: cma
            pres[str(len(list(season))+1)] = [910.0, 925.0] #Mangkhut
            wind[str(len(list(season))+1)] = [65*1.944, 58*1.944] #Change m/s to knots
            coord[str(len(list(season))+1)] = np.array([[17.8,123.4],[18.1,121.5]])    
                

    elif area == 1: #GD
        if source == 0: #jtwc
            if after2014 == True:
                tcindex = landfallTC_GD_jtwc_after2014
                posindex = GD_TCposition_jtwc_after2014
            else:
                tcindex = landfallTC_GD_jtwc
                posindex = GD_TCposition_jtwc
        elif source == 1: #cma
            if after2014 == True:
                tcindex = landfallTC_GD_cma_after2014
                posindex = GD_TCposition_cma_after2014
            else:
                tcindex = landfallTC_GD_cma
                posindex = GD_TCposition_cma
        elif source == 2: #wmo
            if after2014 == True:
                tcindex = landfallTC_GD_wmo_after2014
                posindex = GD_TCposition_wmo_after2014
            else:
                tcindex = landfallTC_GD_wmo
                posindex = GD_TCposition_wmo

        for i in list(tcindex):
            distlist = DIST2LAND[int(i)][DIST2LAND[int(i)]>=0]
            landindexes = np.where(distlist == 0)
            landindexinarea = np.intersect1d(posindex[str(i)],landindexes)

            if landindexinarea.size != 0:
                first_land_index = landindexinarea[0]
                if first_land_index == 0:
                    indexlist = [first_land_index]
                else: 
                    indexlist = [first_land_index-1, first_land_index]

                pres[str(i)] = MIN_PRES[:,:,source][int(i)][indexlist]
                pres[str(i)] = [k for k in pres[str(i)] if k >= 0] #Remove non-existant data
                if pres[str(i)] == []:
                    del pres[str(i)]
                wind[str(i)] = MAX_WIND[:,:,source][int(i)][indexlist]
                wind[str(i)] = [k for k in wind[str(i)] if k >= 0]
                if wind[str(i)] == []:
                    del wind[str(i)]
                coord[str(i)] = np.array([LAT[:,:,source][int(i),indexlist],LON[:,:,source][int(i),indexlist]]).transpose()
                if list(coord[str(i)][0]) == [-3e4,-3e4]: #Remove non-existant data
                    coord[str(i)] = np.delete(coord[str(i)],0,0)
                disttoland[str(i)] = DIST2LAND[int(i)][indexlist]
                list_of_index[str(i)] = indexlist
        
        if source == 0:
            #Add Hato (to the end of the dict), source: jtwc
            pres[str(len(list(season)))] = [956.0, 964.0] #Hato
            wind[str(len(list(season)))] = [90., 85.] #In knots
            coord[str(len(list(season)))] = np.array([[21.5,114.5],[22.2,112.8]])
            
        elif source == 1:        
            #Add Hato and Mangkhut (to the end of the dict), source: cma
            pres[str(len(list(season)))] = [950.0, 950.0] #Hato
            wind[str(len(list(season)))] = [42*1.944, 45*1.944] #Change m/s to knots
            coord[str(len(list(season)))] = np.array([[21.5,114.7],[22.0,113.2]])

            pres[str(len(list(season))+1)] = [950.0, 960.0] #Mangkhut
            wind[str(len(list(season))+1)] = [48*1.944, 42*1.944]
            coord[str(len(list(season))+1)] = np.array([[21.4, 113.8],[21.9, 112.0]])
            
    return pres, wind, coord, disttoland, list_of_index
            
##################### READ TILL HERE #################
    
landfalldata_pres_PHI_jtwc, landfalldata_wind_PHI_jtwc, landfalldata_coord_PHI_jtwc, landfalldata_dist2land_PHI_jtwc, landfalldata_indexlist_PHI_jtwc = get_landfall_data_beforeandafter(0,0)
landfalldata_pres_GD_jtwc, landfalldata_wind_GD_jtwc, landfalldata_coord_GD_jtwc, landfalldata_dist2land_GD_jtwc, landfalldata_indexlist_GD_jtwc = get_landfall_data_beforeandafter(1,0)
landfalldata_pres_PHI2014_jtwc, landfalldata_wind_PHI2014_jtwc, landfalldata_coord_PHI2014_jtwc, landfalldata_dist2land_PHI2014_jtwc, landfalldata_indexlist_PHI2014_jtwc = get_landfall_data_beforeandafter(0,0,True)
landfalldata_pres_GD2014_jtwc, landfalldata_wind_GD2014_jtwc, landfalldata_coord_GD2014_jtwc, landfalldata_dist2land_GD2014_jtwc, landfalldata_indexlist_GD2014_jtwc = get_landfall_data_beforeandafter(1,0,True)

landfalldata_pres_PHI_cma, landfalldata_wind_PHI_cma, landfalldata_coord_PHI_cma, landfalldata_dist2land_PHI_cma, landfalldata_indexlist_PHI_cma = get_landfall_data_beforeandafter(0,1)
landfalldata_pres_GD_cma, landfalldata_wind_GD_cma, landfalldata_coord_GD_cma, landfalldata_dist2land_GD_cma, landfalldata_indexlist_GD_cma = get_landfall_data_beforeandafter(1,1)
landfalldata_pres_PHI2014_cma, landfalldata_wind_PHI2014_cma, landfalldata_coord_PHI2014_cma, landfalldata_dist2land_PHI2014_cma, landfalldata_indexlist_PHI2014_cma = get_landfall_data_beforeandafter(0,1,True)
landfalldata_pres_GD2014_cma, landfalldata_wind_GD2014_cma, landfalldata_coord_GD2014_cma, landfalldata_dist2land_GD2014_cma, landfalldata_indexlist_GD2014_cma = get_landfall_data_beforeandafter(1,1,True)

landfalldata_pres_PHI_wmo, landfalldata_wind_PHI_wmo, landfalldata_coord_PHI_wmo, landfalldata_dist2land_PHI_wmo, landfalldata_indexlist_PHI_wmo = get_landfall_data_beforeandafter(0,2)
landfalldata_pres_GD_wmo, landfalldata_wind_GD_wmo, landfalldata_coord_GD_wmo, landfalldata_dist2land_GD_wmo, landfalldata_indexlist_GD_wmo = get_landfall_data_beforeandafter(1,2)
landfalldata_pres_PHI2014_wmo, landfalldata_wind_PHI2014_wmo, landfalldata_coord_PHI2014_wmo, landfalldata_dist2land_PHI2014_wmo, landfalldata_indexlist_PHI2014_wmo = get_landfall_data_beforeandafter(0,2,True)
landfalldata_pres_GD2014_wmo, landfalldata_wind_GD2014_wmo, landfalldata_coord_GD2014_wmo, landfalldata_dist2land_GD2014_wmo, landfalldata_indexlist_GD2014_wmo = get_landfall_data_beforeandafter(1,2,True)

landfalldata_pres_PHI_N_jtwc, landfalldata_wind_PHI_N_jtwc, landfalldata_coord_PHI_N_jtwc, landfalldata_dist2land_PHI_N_jtwc, landfalldata_indexlist_PHI_N_jtwc = get_landfall_data_beforeandafter(0,0,False,'N')
landfalldata_pres_PHI_N2014_jtwc, landfalldata_wind_PHI_N2014_jtwc, landfalldata_coord_PHI_N2014_jtwc, landfalldata_dist2land_PHI_N2014_jtwc, landfalldata_indexlist_PHI_N2014_jtwc = get_landfall_data_beforeandafter(0,0,True,'N')
landfalldata_pres_PHI_S_jtwc, landfalldata_wind_PHI_S_jtwc, landfalldata_coord_PHI_S_jtwc, landfalldata_dist2land_PHI_S_jtwc, landfalldata_indexlist_PHI_S_jtwc = get_landfall_data_beforeandafter(0,0,False,'S')
landfalldata_pres_PHI_S2014_jtwc, landfalldata_wind_PHI_S2014_jtwc, landfalldata_coord_PHI_S2014_jtwc, landfalldata_dist2land_PHI_S2014_jtwc, landfalldata_indexlist_PHI_S2014_jtwc = get_landfall_data_beforeandafter(0,0,True,'S')

landfalldata_pres_PHI_N_cma, landfalldata_wind_PHI_N_cma, landfalldata_coord_PHI_N_cma, landfalldata_dist2land_PHI_N_cma, landfalldata_indexlist_PHI_N_cma = get_landfall_data_beforeandafter(0,1,False,'N')
landfalldata_pres_PHI_N2014_cma, landfalldata_wind_PHI_N2014_cma, landfalldata_coord_PHI_N2014_cma, landfalldata_dist2land_PHI_N2014_cma, landfalldata_indexlist_PHI_N2014_cma = get_landfall_data_beforeandafter(0,1,True,'N')
landfalldata_pres_PHI_S_cma, landfalldata_wind_PHI_S_cma, landfalldata_coord_PHI_S_cma, landfalldata_dist2land_PHI_S_cma, landfalldata_indexlist_PHI_S_cma = get_landfall_data_beforeandafter(0,1,False,'S')
landfalldata_pres_PHI_S2014_cma, landfalldata_wind_PHI_S2014_cma, landfalldata_coord_PHI_S2014_cma, landfalldata_dist2land_PHI_S2014_cma, landfalldata_indexlist_PHI_S2014_cma = get_landfall_data_beforeandafter(0,1,True,'S')

landfalldata_pres_PHI_N_wmo, landfalldata_wind_PHI_N_wmo, landfalldata_coord_PHI_N_wmo, landfalldata_dist2land_PHI_N_wmo, landfalldata_indexlist_PHI_N_wmo = get_landfall_data_beforeandafter(0,2,False,'N')
landfalldata_pres_PHI_N2014_wmo, landfalldata_wind_PHI_N2014_wmo, landfalldata_coord_PHI_N2014_wmo, landfalldata_dist2land_PHI_N2014_wmo, landfalldata_indexlist_PHI_N2014_wmo = get_landfall_data_beforeandafter(0,2,True,'N')
landfalldata_pres_PHI_S_wmo, landfalldata_wind_PHI_S_wmo, landfalldata_coord_PHI_S_wmo, landfalldata_dist2land_PHI_S_wmo, landfalldata_indexlist_PHI_S_wmo = get_landfall_data_beforeandafter(0,2,False,'S')
landfalldata_pres_PHI_S2014_wmo, landfalldata_wind_PHI_S2014_wmo, landfalldata_coord_PHI_S2014_wmo, landfalldata_dist2land_PHI_S2014_wmo, landfalldata_indexlist_PHI_S2014_wmo = get_landfall_data_beforeandafter(0,2,True,'S')

toc = time.time()
print(toc - tic)

In [ ]:
#Calculate Pressure deficit

def pressure_deficit(area,source,separate_PHI = False, intersect = False):
    #separate_PHI: 'N','S' or False
    #intersect: 'True' if only using data of TCs appearing in all 3 agencies (to compare like to like events), 'False' otherwise
    
    pressure_dict = get_landfall_data_beforeandafter(area,source,False,separate_PHI)[0]
    pres_def_before_list = []
    pres_def_after_list = []
    
    if intersect == True:
        key_a = list(get_landfall_data_beforeandafter(area,0,False,separate_PHI)[0].keys())
        key_b = list(get_landfall_data_beforeandafter(area,1,False,separate_PHI)[0].keys())
        key_c = list(get_landfall_data_beforeandafter(area,2,False,separate_PHI)[0].keys())
        key_intersect_list = np.intersect1d(np.intersect1d(key_a,key_b),key_c) #Intersected list of TCs
    else:
        key_intersect_list = list(pressure_dict.keys())
    
    for key in key_intersect_list:
        if len(pressure_dict[key]) == 1: 
            pres_def_after = 1010 - pressure_dict[key][0]
            pres_def_after_list.append(pres_def_after)
        else: 
            pres_def_before = 1010 - pressure_dict[key][0]
            pres_def_after = 1010 - pressure_dict[key][1]
            pres_def_before_list.append(pres_def_before)
            pres_def_after_list.append(pres_def_after)
    return pres_def_before_list, pres_def_after_list #pressure_deficit(area)[0] for the first list and pressure_deficit(area)[1] for the second list

In [ ]:
#Calculate Pressure deficit

def pressure_deficit(area,source,separate_PHI = False, intersect = False):
    #separate_PHI: 'N','S' or False
    #intersect: 'True' if only using data of TCs appearing in all 3 agencies (to compare like to like events), 'False' otherwise
    
    pressure_dict = get_landfall_data_beforeandafter(area,source,False,separate_PHI)[0]
    pres_def_before_list = []
    pres_def_after_list = []
    
    if intersect == True:
        key_a = list(get_landfall_data_beforeandafter(area,0,False,separate_PHI)[0].keys())
        key_b = list(get_landfall_data_beforeandafter(area,1,False,separate_PHI)[0].keys())
        key_c = list(get_landfall_data_beforeandafter(area,2,False,separate_PHI)[0].keys())
        key_intersect_list = np.intersect1d(np.intersect1d(key_a,key_b),key_c) #Intersected list of TCs
    else:
        key_intersect_list = list(pressure_dict.keys())
    
    for key in key_intersect_list:
        if len(pressure_dict[key]) == 1: 
            pres_def_after = 1010 - pressure_dict[key][0]
            pres_def_after_list.append(pres_def_after)
        else: 
            pres_def_before = 1010 - pressure_dict[key][0]
            pres_def_after = 1010 - pressure_dict[key][1]
            pres_def_before_list.append(pres_def_before)
            pres_def_after_list.append(pres_def_after)
    return pres_def_before_list, pres_def_after_list #pressure_deficit(area)[0] for the first list and pressure_deficit(area)[1] for the second list

In [ ]:
tic = time.time()
# presdef_probability(area=0,source=0,binsize=10)
# presdef_probability(0,1,10)
# presdef_probability(0,2,10)
# presdef_probability(1,0,10, False,False)
# presdef_probability(1,1,10, False,False)
# presdef_probability(1,2,10, False,False)
# presdef_probability(0,0,10,'N')
# presdef_probability(0,0,10,'S')
# presdef_probability(0,1,10,'N')
# presdef_probability(0,1,10,'S')
#presdef_probability(0,2,10,'N')
#presdef_probability(0,2,10,'S')
toc = time.time()
print(toc-tic)

In [ ]:
#----------------------------------------------------
'''
calculating probability of max wind speed with log-normal fit
'''
def wind_speed(area, source, separate_PHI=False, intersect = False, pd_wind_int = False):
    #pd_wind_int: 'True' if same TCs for plotting pd prob (in above) and wind prob, 'False' otherwise
    windspeed_dict = get_landfall_data_beforeandafter(area, source, False,separate_PHI)[1]
    windspeed_before_list = []
    windspeed_after_list = []
    
    if pd_wind_int == True:
        pressure_dict = get_landfall_data_beforeandafter(area, source, False,separate_PHI)[0]
        key_a = list(pressure_dict.keys())
        key_b = list(windspeed_dict.keys())
        key_intersect_list = np.intersect1d(key_a,key_b)
    
    elif intersect == True:
        key_a = list(get_landfall_data_beforeandafter(area,0,False,separate_PHI)[1].keys())
        key_b = list(get_landfall_data_beforeandafter(area,1,False,separate_PHI)[1].keys())
        key_c = list(get_landfall_data_beforeandafter(area,2,False,separate_PHI)[1].keys())
        key_intersect_list = np.intersect1d(np.intersect1d(key_a,key_b),key_c)
    else:
        key_intersect_list = list(windspeed_dict.keys())
        
    
    for key in key_intersect_list:
        if len(windspeed_dict[key]) == 1: 
            windspeed_after_list. append(windspeed_dict[key][0])
        else: 
            windspeed_before_list.append(windspeed_dict[key][0])
            windspeed_after_list.append(windspeed_dict[key][1])

    return windspeed_before_list, windspeed_after_list #before landfall, after landfall

In [ ]:
#----------------------------------------------------
'''
calculating probability of max wind speed 
'''
def wind_probability(area, source, separate_PHI = False, fitnorm = False, intersect = False, pd_wind_int = False):
    #fitnorm: 'True' if fitting normal function, 'False' if fitting log-normal
    x_range = list(range(0,131,10))
    b, a = wind_speed(area, source, separate_PHI, intersect, pd_wind_int)
    b = [i for i in b if i != 0] #Remove 0 values as they are actually missing data
    a = [j for j in a if j != 0]
    before_list = [int(i) for i in b]
    after_list = [int(j) for j in a]
    

    
    prob_before = []
    prob_after = []
    for i in range(len(x_range)-1) : 
        count_before = 0 
        count_after = 0
        for j in range(len(after_list)):
            if i < len(x_range)-1 and x_range[i+1] > after_list[j] >= x_range[i]: # to make sure that it only includes one range each time but not more than one
                count_after = count_after + 1 
        for j in range(len(before_list)):
            if i < len(x_range)-1 and x_range[i+1] > before_list[j] >= x_range[i]:
                count_before = count_before + 1 
        prob_before.append((count_before/len(before_list))/(130/len(x_range)))
        prob_after.append((count_after/len(after_list))/(130/len(x_range)))
        
    prob_before = np.asarray(prob_before)
    prob_after = np.asarray(prob_after)
        
    #Plot before landfall
    plt.figure()
    plt.xlabel('Maximum Wind Speed (kt)')
    plt.ylabel('Probability') 
    plt.ylim(0,0.06)
    x_axis = list(np.arange(5,130,10))
    plt.scatter(x_axis, prob_before,color = 'k')

    x = np.arange(0, 131, 1)
    
    if fitnorm == True:
        mu, std = norm.fit(after_list)
        p = norm.pdf(x, mu,std)
        skew = 0
    else:
        s, loc, scale = lognorm.fit(before_list, loc = 0)
        mu, var, skew = lognorm.stats(s, loc, scale, moments='mvs')
        std = np.sqrt(var)
        p = lognorm.pdf(x, s, loc, scale)
    p = np.asarray(p)
    zeroindexlist = np.where(p == 0)
    p[zeroindexlist] = 1e-80 #Make zero to a very small number for chi-square test
    
    x_at_datapt_index = [i for i in range(len(x)) if x[i] in x_axis] #Find index in x where there is a data point in x_axis
    p_mse = p[x_at_datapt_index]
    chisq, pvalue = chisquare(prob_before*130/len(x_range)*len(before_list), p_mse*130/len(x_range)*len(before_list), ddof=2) #Perform chi-sq test
#    print(str(source), chisq, pvalue)

    rmserr = np.sqrt(mean_squared_error(prob_before, p_mse))
    
    samplesize = len(before_list)
    plt.plot(x, p,'b-')
    
    #Plot title
    if area == 0:
        if source == 0:
            if separate_PHI == False:
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
            elif separate_PHI == 'N':
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Northern Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
            elif separate_PHI == 'S':
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Southern Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
        elif source == 1:
            if separate_PHI == False:
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
            elif separate_PHI == 'N':
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Northern Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
            elif separate_PHI == 'S':
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Southern Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
        elif source == 2:
            if separate_PHI == False:
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
            elif separate_PHI == 'N':
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Northern Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
            elif separate_PHI == 'S':
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Southern Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
    
    else: 
        if source == 0:
            plt.title('Probability distribution of max wind speed of typhoons before their landfalls in Guangdong \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
        elif source == 1:
            plt.title('Probability distribution of max wind speed of typhoons before their landfalls in Guangdong \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
        elif source == 2:
            plt.title('Probability distribution of max wind speed of typhoons before their landfalls in Guangdong \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
    plt.show()
    
    
    #Plot after landfall
    plt.figure()
    plt.xlabel('Maximum Wind Speed (kt)')  
    plt.ylabel('Probability') 
    plt.ylim(0,0.06)
    x_axis = list(np.arange(5,130,10))
    plt.scatter(x_axis, prob_after,color = 'r')
    
    x = np.arange(0, 131, 1)
    if fitnorm == True:
        mu, std = norm.fit(after_list)
        p = norm.pdf(x, mu,std)
        skew = 0
    else:
        s, loc, scale = lognorm.fit(after_list, loc = 0)
        mu, var, skew = lognorm.stats(s, loc, scale, moments='mvs')
        std = np.sqrt(var)
        p = lognorm.pdf(x, s, loc, scale)
    p = np.asarray(p)
    zeroindexlist = np.where(p == 0)
    p[zeroindexlist] = 1e-80 #Make zero to a very small number for chi-square test
    
    x_at_datapt_index = [i for i in range(len(x)) if x[i] in x_axis] #Find index in x where there is a data point in x_axis
    p_mse = p[x_at_datapt_index]
    chisq, pvalue = chisquare(prob_after*130/len(x_range)*len(after_list), p_mse*130/len(x_range)*len(after_list), ddof=2)
#    print(str(source), chisq, pvalue)

    rmserr = np.sqrt(mean_squared_error(prob_after, p_mse))

    samplesize = len(after_list)
    plt.plot(x, p, 'b-')
    
    print(prob_after*130/len(x_range)*len(after_list))
    print(p_mse*130/len(x_range)*len(after_list))

    
    if area == 0:
        if source == 0:
            if separate_PHI == False:
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
            elif separate_PHI == 'N':
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Northern Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
            elif separate_PHI == 'S':
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Southern Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
        elif source == 1:
            if separate_PHI == False:
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
            elif separate_PHI == 'N':
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Northern Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
            elif separate_PHI == 'S':
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Southern Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
        elif source == 2: 
            if separate_PHI == False:
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
            elif separate_PHI == 'N':
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Northern Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
            elif separate_PHI == 'S':
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Southern Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
    else: 
        if source == 0:
            plt.title('Probability distribution of max wind speed of typhoons at their landfalls in Guangdong \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
        elif source == 1:
            plt.title('Probability distribution of max wind speed of typhoons at their landfalls in Guangdong \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
        elif source == 2:
            plt.title('Probability distribution of max wind speed of typhoons at their landfalls in Guangdong \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
    plt.show()
    
    return prob_before, prob_after

In [ ]:
tic = time.time()
wind_probability(0,0,False)
# wind_probability(0,1,False)
# wind_probability(0,2,False)
# wind_probability(1,0,False,False,False,True)
# wind_probability(1,0,False,False,False,False)
# wind_probability(1,1,False,False,False,True)
# wind_probability(1,2,False,False,False,True)
# wind_probability(0, 0,'N')
# wind_probability(0, 1,'N')
# wind_probability(0, 2,'N')
# wind_probability(0, 0,'S')
# wind_probability(0, 1,'S')
# wind_probability(0, 2,'S')
toc = time.time()
print(toc-tic)

In [ ]:
#Visualise TC coordinates
#Plots the coastline and the landfall TC coordinates of whole area
def plot_points(area, source, after2014 = False, PHI_NS = False): 
    if area == 0: #PHI
        landcoord = z_land_coord_PHI
        if source == 0:
            if after2014 == True:
                if PHI_NS == False:
                    coordlist = landfalldata_coord_PHI2014_jtwc #List of coordinates of TC centre
                    keys = landfalldata_coord_PHI2014_jtwc.keys() #Keys of dict (TC indexes)
                elif PHI_NS == 'N':
                    coordlist = landfalldata_coord_PHI_N2014_jtwc
                    keys = landfalldata_coord_PHI_N2014_jtwc.keys()
                elif PHI_NS == 'S':
                    coordlist = landfalldata_coord_PHI_S2014_jtwc
                    keys = landfalldata_coord_PHI_S2014_jtwc.keys()
            elif after2014 == False:
                if PHI_NS == False:
                    coordlist = landfalldata_coord_PHI_jtwc
                    keys = landfalldata_coord_PHI_jtwc.keys()
                elif PHI_NS == 'N':
                    coordlist = landfalldata_coord_PHI_N_jtwc
                    keys = landfalldata_coord_PHI_N_jtwc.keys()
                elif PHI_NS == 'S':
                    coordlist = landfalldata_coord_PHI_S_jtwc
                    keys = landfalldata_coord_PHI_S_jtwc.keys()
                    
        elif source == 1:
            if after2014 == True:
                if PHI_NS == False:
                    coordlist = landfalldata_coord_PHI2014_cma
                    keys = landfalldata_coord_PHI2014_cma.keys()
                elif PHI_NS == 'N':
                    coordlist = landfalldata_coord_PHI_N2014_cma
                    keys = landfalldata_coord_PHI_N2014_cma.keys()
                elif PHI_NS == 'S':
                    coordlist = landfalldata_coord_PHI_S2014_cma
                    keys = landfalldata_coord_PHI_S2014_cma.keys()
            elif after2014 == False:
                if PHI_NS == False:
                    coordlist = landfalldata_coord_PHI_cma
                    keys = landfalldata_coord_PHI_cma.keys()
                elif PHI_NS == 'N':
                    coordlist = landfalldata_coord_PHI_N_cma
                    keys = landfalldata_coord_PHI_N_cma.keys()
                elif PHI_NS == 'S':
                    coordlist = landfalldata_coord_PHI_S_cma
                    keys = landfalldata_coord_PHI_S_cma.keys()
        elif source == 2:
            if after2014 == True:
                if PHI_NS == False:
                    coordlist = landfalldata_coord_PHI2014_wmo
                    keys = landfalldata_coord_PHI2014_wmo.keys()
                elif PHI_NS == 'N':
                    coordlist = landfalldata_coord_PHI_N2014_wmo
                    keys = landfalldata_coord_PHI_N2014_wmo.keys()
                elif PHI_NS == 'S':
                    coordlist = landfalldata_coord_PHI_S2014_wmo
                    keys = landfalldata_coord_PHI_S2014_wmo.keys()
            elif after2014 == False:
                if PHI_NS == False:
                    coordlist = landfalldata_coord_PHI_wmo
                    keys = landfalldata_coord_PHI_wmo.keys()
                elif PHI_NS == 'N':
                    coordlist = landfalldata_coord_PHI_N_wmo
                    keys = landfalldata_coord_PHI_N_wmo.keys()
                elif PHI_NS == 'S':
                    coordlist = landfalldata_coord_PHI_S_wmo
                    keys = landfalldata_coord_PHI_S_wmo.keys()
                    
                    
                    
    elif area == 1: #GD
        landcoord = z_land_coord_GD
        if source == 0:
            if after2014 == True:
                coordlist = landfalldata_coord_GD2014_jtwc
                keys = landfalldata_coord_GD2014_jtwc.keys()
            elif after2014 == False:
                coordlist = landfalldata_coord_GD_jtwc
                keys = landfalldata_coord_GD_jtwc.keys()
        elif source == 1:
            if after2014 == True:
                coordlist = landfalldata_coord_GD2014_cma
                keys = landfalldata_coord_GD2014_cma.keys()
            elif after2014 == False:
                coordlist = landfalldata_coord_GD_cma
                keys = landfalldata_coord_GD_cma.keys()
        elif source == 2:
            if after2014 == True:
                coordlist = landfalldata_coord_GD2014_wmo
                keys = landfalldata_coord_GD2014_wmo.keys()
            elif after2014 == False:
                coordlist = landfalldata_coord_GD_wmo
                keys = landfalldata_coord_GD_wmo.keys()    
    color = iter(cm.rainbow(np.linspace(0,1,len(keys))))
    plt.figure()  
    if area == 0: 
        if source == 0:
            if PHI_NS == False:
                plt.title('Tracks of typhoons with landfall in the Philippines, source: JTWC') 
            elif PHI_NS == 'N':
                plt.title('Tracks of typhoons with landfall in the Northern Philippines, source: JTWC') 
            elif PHI_NS == 'S':
                plt.title('Tracks of typhoons with landfall in the Southern Philippines, source: JTWC')
        elif source == 1:
            if PHI_NS == False:
                plt.title('Tracks of typhoons with landfall in the Philippines, source: CMA') 
            elif PHI_NS == 'N':
                plt.title('Tracks of typhoons with landfall in the Northern Philippines, source: CMA') 
            elif PHI_NS == 'S':
                plt.title('Tracks of typhoons with landfall in the Southern Philippines, source: CMA')
        elif source == 2:
            if PHI_NS == False:
                plt.title('Tracks of typhoons with landfall in the Philippines, source: WMO') 
            elif PHI_NS == 'N':
                plt.title('Tracks of typhoons with landfall in the Northern Philippines, source: WMO') 
            elif PHI_NS == 'S':
                plt.title('Tracks of typhoons with landfall in the Southern Philippines, source: WMO')
    else:
        if source == 0:
            plt.title('Tracks of typhoons with landfall in Guangdong, source: JTWC')
        elif source == 1:
            plt.title('Tracks of typhoons with landfall in Guangdong, source: CMA')
        elif source == 2:   
            plt.title('Tracks of typhoons with landfall in Guangdong, source: WMO')
    plt.xlabel('Longitude (E)') 
    plt.ylabel('Latitude (N)')
    plt.scatter(landcoord[:,0],landcoord[:,1],color = 'k', s=5)
    if after2014 == True:
        size = 20
    else:
        size = 5
    for index in list(keys):
        c=next(color)
        plt.scatter(coordlist[index][:,1],coordlist[index][:,0],color = c,s=size, label = index)
    if after2014 == False:
        plt.legend(fontsize=4)
    else:
        plt.legend(fontsize='x-small')
    plt.axis('equal')


#Plots the coastline and the landfall TC coordinates of individual TC
def plot_points_landfall_indiv(source, tcindex):
    plt.figure()
    plt.title(str(tcindex))
    if source == 0:
        coorddictPHI = landfalldata_coord_PHI_jtwc
        coorddictGD = landfalldata_coord_GD_jtwc
    else:
        coorddictPHI = landfalldata_coord_PHI_cma
        coorddictGD = landfalldata_coord_GD_cma
    if str(tcindex) in landfalldata_coord_PHI:
        plt.scatter(z_land_coord_PHI[:,0],z_land_coord_PHI[:,1],color = 'k')
        plt.scatter(coorddictPHI[str(tcindex)][:,1],coorddictPHI[str(tcindex)][:,0], color = 'r')
    if str(tcindex) in landfalldata_coord_GD:
        plt.scatter(z_land_coord_GD[:,0],z_land_coord_GD[:,1],color = 'k')
        plt.scatter(coorddictGD[str(tcindex)][:,1],coorddictGD[str(tcindex)][:,0], color = 'b')

In [ ]:
tic = time.time()
plot_points(0, 0, True, False)
plot_points(0, 1, True, False)
plot_points(0, 2, True, False)
plot_points(0, 0, True, 'N')
plot_points(0, 1, True, 'N')
plot_points(0, 0, True, 'S')
plot_points(0, 1, True, 'S')
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()
plot_points(0, 0, False)
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()
plot_points(1, 1, True)
#plot_points(1, 1, True)
#plot_points(1, 0, False)
#plot_points(1, 1, False)
toc = time.time()
print(toc-tic)

In [ ]:
#------------------------------------------------------
'''
extract dates of typhoon for rainfall data
'''

time_units = ibt.variables['source_time'].getncattr('units')

#Change dates of TCs to common understandable format
def getdates(tcindex): 
    datelist = list(num2date(time_record,time_units)[tcindex])

    return datelist #returns a list in (year, month, day, hour, minute, second) format

#Get dates of TCs at landfall
def getdates_landfall(area, after2014 = False):
    landfall_dates = {}
    if after2014 == True:
        indexlist = get_landfall_data_beforeandafter(area, True)[4]
    else:
        indexlist = get_landfall_data_beforeandafter(area, False)[4]
        
    for key in indexlist.keys():
        print(key)
        landfall_dates[key] = list(num2date(time_record,time_units)[int(key), indexlist[key]])
    return landfall_dates

#WARNING: Extremely long runtime, dates are pasted in cell below so code does not have to be rerun again

In [ ]:
# ------------------------------------------------------
'''
Probability of landfall in GD and PHI 
'''
tic = time.time() #WARNING: Expected runtime ~30min

#Creates a dictionary recording data
def landfall_probability(area, source):
    yeardict = {}
    yeardicttotal = {}
    
    if source == 0:
        startyear = 1945
    elif source == 1:
        startyear = 1949
    elif source == 2: 
        startyear = 1951
        
    for i in range(startyear, 2017): #No data before startyear, and 2017 data not complete
        yeardict[str(i)] = []
        yeardicttotal[str(i)] = []
    for tcindex in range(len(list(season))):
        if tcindex%100 == 0: #Just to check whether code is running + progress
            print(tcindex)
        i = list(season)[tcindex] #i is the year
        if i in list(range(startyear,2017)):
            yeardicttotal[str(i)].append(tcindex)
            if area == 0:
                if source == 0:
                    if str(tcindex) in landfalldata_coord_PHI_jtwc.keys():
                        yeardict[str(i)].append(tcindex)
                elif source == 1:
                    if str(tcindex) in landfalldata_coord_PHI_cma.keys():
                        yeardict[str(i)].append(tcindex)
                elif source == 2:
                    if str(tcindex) in landfalldata_coord_PHI_wmo.keys():
                        yeardict[str(i)].append(tcindex)
            else:
                if source == 0:
                    if str(tcindex) in landfalldata_coord_GD_jtwc.keys():
                        yeardict[str(i)].append(tcindex)
                elif source == 1:
                    if str(tcindex) in landfalldata_coord_GD_cma.keys():
                        yeardict[str(i)].append(tcindex)
                elif source == 2:
                    if str(tcindex) in landfalldata_coord_GD_wmo.keys():
                        yeardict[str(i)].append(tcindex)

    landfall_number_list = []
    landfall_number_list_total = []
    landfall_counts = {}
    landfall_counts_total = {}
    for year in yeardict.keys():
        landfall_number_list.append(len(yeardict[year])) #Append the number of landfalls in year
#        landfall_number_list_total.append(len(yeardicttotal[year]))
    for n in range(max(landfall_number_list)+1):
        landfall_counts[str(n)] = landfall_number_list.count(n)
#        landfall_counts_total[str(n)] = landfall_number_list_total.count(n)
        
#     print(yeardict, landfall_number_list)
    return landfall_counts #, yeardict, yeardicttotal
    
landfall_no_PHI_jtwc = landfall_probability(0,0)
landfall_no_PHI_cma = landfall_probability(0,1)
landfall_no_PHI_wmo = landfall_probability(0,2)
landfall_no_GD_jtwc = landfall_probability(1,0)
landfall_no_GD_cma = landfall_probability(1,1)
landfall_no_GD_wmo = landfall_probability(1,2)

toc = time.time()
print(toc-tic)